In [2]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
data = requests.get('https://en.wikipedia.org/wiki/List_of_serial_killers_in_the_United_States')

In [4]:
soup = BeautifulSoup(data.content, 'html.parser')

In [5]:
sk_table=soup.find_all("table")[0]

In [6]:
rows = sk_table.findChildren("tr")

In [7]:
data = []
# thanks Stack Overflow: https://stackoverflow.com/questions/23377533/python-beautifulsoup-parsing-table
rows = sk_table.find_all('tr')[1:]
for row in rows:
    urls = row.find_all('a', href=True)
    if urls != []:
        sk_url = urls[0]['href']
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    cols.append(sk_url)
    data.append([ele for ele in cols if ele])

In [8]:
### indexing: name, years active, proven victims, possible victims, status, reference (not useful in our case), wikipedia reference page

In [9]:
# need to filter out the serial killers who were active before 1980

In [10]:
filtered_entries = []
unknown = []
for entry in data:
    years_active = entry[1]
    if "–" in years_active:
        year_split = years_active.split("–")
        start = year_split[0].strip()
        end = year_split[1].strip()
        try:
            if int(end) >= 1980:
                filtered_entries.append(entry)
        except:
            unknown.append(entry)
    else:
        try: 
            if int(years_active) >= 1980:
                filtered_entries.append(entry)
        except:
            unknown.append(entry)

In [11]:
unknown

[['Eaton, Dale Wayne',
  '1988, 2001 (known)  c. 1983-1996 (suspected)',
  '2',
  '11+',
  'Sentenced to death (overturned - awaiting resentencing hearing)',
  'Eaton is also suspected of being behind the killings known as the "Great Basin Murders".',
  '[110]',
  '/wiki/Lil%27_Miss_murder'],
 ['Falls, Neal',
  '?–2015?',
  '0',
  '10+',
  'Killed by intended victim',
  'Suspected of killing up to 10 women before being killed in self defense.',
  '[123]',
  '/wiki/Neal_Falls'],
 ['Fisher, Lavinia',
  '?–1819?"',
  '0',
  'Numerous',
  'Executed 1820',
  'Along with her husband John was a member of a highway gang; allegedly killed travelers visiting her inn.',
  '[127]',
  '/wiki/Lavinia_Fisher'],
 ['Halliday, Lizzie',
  '1890s',
  '4',
  '7',
  'Died in Matteawan State Hospital for the Criminally Insane',
  'First woman sentenced to be executed by the electric chair',
  '[177]',
  '/wiki/Lizzie_Halliday'],
 ['Johnson, John',
  '1843–?',
  '0',
  '300+',
  'Died 1900',
  'Known as "Live

In [12]:
filtered_entries.append(unknown[0])
filtered_entries.append(unknown[1])

In [13]:
table_scrape = pd.DataFrame(filtered_entries)

In [14]:
table_scrape

,0,1,2,3,4,5,6,7
0,"Ables, Tony",1970–1990,2,4+,Sentenced to death; commuted to life imprisonment,"Murdered robbery victim in 1970, and at least ...",[4],/wiki/Tony_Ables
1,"Ackroyd, John Arthur",1976–1992,1,3+,Died in prison,Suspected in multiple murders along Oregon's H...,[5],/wiki/John_Arthur_Ackroyd
2,"Allen, Howard",1974–1987,3,3,Incarcerated for 60 years,Death Sentence Commuted,[8][9],/wiki/Howard_Allen
3,"Allen, Quincy",2002,4,4,Sentenced to death,Murdered four people in North and South Caroli...,[10],/wiki/Quincy_Allen
4,"Angelo, Richard",1987,4,10,Sentenced to 50 years to life in prison,Long Island nurse who poisoned patients in his...,[11],/wiki/Richard_Angelo
...,...,...,...,...,...,...,...,...
246,"Wright, Douglas",1969–1991,5,5+,Executed 1996,First criminal executed by lethal injection in...,[425],/wiki/Douglas_Wright_(murderer)
247,"Wuornos, Aileen",1989–1990,7,7,Executed 2002,"Also known as the ""Damsel of Death""; shot seve...",[426],/wiki/Aileen_Wuornos
248,"Yates, Robert Lee",1975–1998,13,18+,"Sentenced to death, commuted to life imprisonment",[427],/wiki/Robert_Lee_Yates,None
249,"Eaton, Dale Wayne","1988, 2001 (known) c. 1983-1996 (suspected)",2,11+,Sentenced to death (overturned - awaiting rese...,Eaton is also suspected of being behind the ki...,[110],/wiki/Lil%27_Miss_murder


In [17]:
murderer_list = table_scrape[0]
url_list = table_scrape[7]

In [27]:
with open("murderer_list.csv", "w+") as f:
    for x in murderer_list:
        f.write(x.replace(",", ""))
        f.write(",")

In [26]:
with open("url_list.csv", "w+") as f:
    for x in url_list:
        if x is not None:
            s = "https://en.wikipedia.org/" + x
            f.write(s)
            f.write(",")

In [14]:
# scrape html from each serial killers webpage
scraped_pages = []
for entry in filtered_entries:
    wiki_url = entry[-1]
    page = requests.get("http://www.wikipedia.org"+wiki_url)
    scraped_pages.append(page.content)

In [149]:
# given html content, extract text information
# source: https://matix.io/extract-text-from-webpage-using-beautifulsoup-and-python/
def get_page_content(page):
    # make soup and get text
    soup = BeautifulSoup(page, 'html.parser')
    text = soup.find_all(text=True)
    output = ''
    blacklist = [
        '[document]',
        'noscript',
        'header',
        'html',
        'meta',
        'head', 
        'input',
        'script',
        '.mw-parser-output'
        'div', 
        'span', 
        'cite',  
        'ul',
        'style',
        'li',
        'ol',
        'form', 
        'body', 
        'div', 
        'b', 
        'body', 
        'h3',
        'h2',
        'label'
    ]
    # filter text output
    for t in text:
        if t.parent.name not in blacklist:
#             print(t.parent.name)
#             print(t)
            output += '{} '.format(t)
    # return scraped text
    #print(output)
    return output

In [150]:
x = get_page_content(scraped_pages[0])

In [151]:
webpage_text = []
for page in scraped_pages:
    content = get_page_content(page)
    webpage_text.append([content])

In [155]:
pages_dict = {}
for i, x in enumerate(webpage_text):
    pages_dict[filtered_entries[i][0]] = x

In [17]:
# now that content has been scraped, need to extract information using natural language processing

In [158]:
import json

with open('wikipedia_serial_killer_scrape.json', 'w') as json_file:
    json.dump(pages_dict, json_file)

In [152]:
print(x)

Tony Ables - Wikipedia Tony Ables Jump to navigation Jump to search template Infobox criminal considered for merging Born December 28, 1954 St. Petersburg, Florida Other names Tony Abels Thadaisis Gordon Michael Jones James Selbey Conviction(s) Murder Criminal penalty Death ; commuted to  life imprisonment Details Victims 4+ (2 convictions) 1970–1990 Country United States Florida June 4, 1990 Imprisoned at Charlotte Correctional Institution ,  Punta Gorda, Florida 
  (born December 28, 1954) is an  American   serial killer . Officially convicted of two murders, one of which occurred while he was still underage, Ables was later connected via  DNA  evidence to two other murders in 2006: of 84-year-old retiree Adeline McLaughlin in 1983, and his 31-year-old girlfriend Deborah Kisor in 1987. Because of their romantic involvement, Ables will not be charged with Kisor's killing, and is yet to be tried to McLaughlin's. [1]  He is currently serving a  life  sentence for killing another girlfri

In [69]:
# source: https://medium.com/@adrianhkay/extract-information-from-news-articles-by-using-web-scraping-and-nlp-7c9cacbae75
import nltk
#nltk.download()
from nltk.corpus import stopwords
stop = stopwords.words('english')
def nltk_process(document):
    document = " ".join([i for i in document.split() if i not in stop])
    sentences = nltk.sent_tokenize(document)
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    return sentences

In [71]:
preprocess_ables = nltk_process(str(webpage_text[0]))

[[('[', 'NN'),
  ('``', '``'),
  ('(', '('),
  ('born', 'JJ'),
  ('December', 'NNP'),
  ('28', 'CD'),
  (',', ','),
  ('1954', 'CD'),
  (')', ')'),
  ('.', '.')],
 [('Officially', 'RB'),
  ('convicted', 'VBN'),
  ('two', 'CD'),
  ('murders', 'NNS'),
  (',', ','),
  ('one', 'CD'),
  ('occurred', 'VBD'),
  ('still', 'RB'),
  ('underage', 'JJ'),
  (',', ','),
  ('Ables', 'NNP'),
  ('later', 'RB'),
  ('connected', 'VBD'),
  ('via', 'IN'),
  ('evidence', 'NN'),
  ('two', 'CD'),
  ('murders', 'NNS'),
  ('2006', 'CD'),
  (':', ':'),
  ('84-year-old', 'JJ'),
  ('retiree', 'NN'),
  ('Adeline', 'NNP'),
  ('McLaughlin', 'NNP'),
  ('1983', 'CD'),
  (',', ','),
  ('31-year-old', 'JJ'),
  ('girlfriend', 'NN'),
  ('Deborah', 'NNP'),
  ('Kisor', 'NNP'),
  ('1987', 'CD'),
  ('.', '.')],
 [('Because', 'IN'),
  ('romantic', 'JJ'),
  ('involvement', 'NN'),
  (',', ','),
  ('Ables', 'NNP'),
  ('charged', 'VBD'),
  ('Kisor', 'NNP'),
  ("'s", 'POS'),
  ('killing', 'NN'),
  (',', ','),
  ('yet', 'RB'),
  ('tr

In [95]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
ps.stem("suffocated")

'suffoc'

In [100]:
potentials = []
for sentence in preprocess_ables:
    for i, word in enumerate(sentence):
        if ps.stem(word[0]) == "murder":
            segment = sentence[i-10:i+11]
            potentials.append(segment)

In [99]:
potentials

[[('He', 'PRP'),
  ('suffocated', 'VBD'),
  ('McLaughlin', 'NNP'),
  ('pillow', 'NN'),
  ('proceeded', 'VBD'),
  ('burglarize', 'JJ'),
  ('apartment', 'NN'),
  ('.', '.')]]

In [25]:
from nltk.stem.porter import PorterStemmer

In [37]:
from nltk.stem import WordNetLemmatizer
wordnet = WordNetLemmatizer()

In [101]:
# lemmas = []
# words = nltk.word_tokenize(str(webpage_text[8]))
# for word in words:
#     l = wordnet.lemmatize(word)
#     lemmas.append(l)
# lemmas

# # get murder phrases
# murder = 

In [72]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [85]:
doc = nlp(str(webpage_text[3]))
for token in doc:
    if token.text == "attack":
        print(token.head.text)
        print([child for child in token.children])

planned
[to, and, kill]


In [102]:
# preprocess to get tokens and stems


In [57]:
import textacy

In [103]:
pattern = r'(<VERB>?<ADV>*<VERB>+)'
#page_doc = textacy.make_spacy_doc(str(webpage_text[0]), lang='en_core_web_sm')
# verb_phrases = textacy.extract.pos_regex_matches(doc, pattern)
# for chunk in verb_phrases:
#     print(chunk.text)

In [104]:
len(webpage_text)

251

In [121]:
from IPython.display import clear_output

In [125]:
def get_murder_info(page):
    for x in page:
        print(x)
        print("\n")
    entries = []
    stop_flag = False
    while not stop_flag:
        entry = input("Enter year, weapon, location: ")
        if entry == "q":
            return entries
        else:
            entries.append(entry)

In [126]:
murder_dict = {}
for i, x in enumerate(webpage_text):
    murders = get_murder_info(x)
    murder_dict[filtered_entries[i][0]] = murders
    clear_output(wait=True)

Tony Ables - Wikipedia 
 
 
 
 
  CentralNotice  
 
 
 Tony Ables 
 
 From Wikipedia, the free encyclopedia 
 
 
 Jump to navigation 
 Jump to search 
 American serial killer 
 ‹ The  template   Infobox criminal  is being  considered for merging . › 
 Tony Ables Inmate mugshot Born Tony Alvin Ables  ( 1954-12-28 )  December 28, 1954  (age 65) St. Petersburg, Florida , U.S. Other names Tony Abels Thadaisis Gordon Michael Jones James Selbey Conviction(s) Murder Criminal penalty Death ; commuted to  life imprisonment Details Victims 4+ (2 convictions) Span of crimes 1970–1990 Country United States State(s) Florida Date apprehended June 4, 1990 Imprisoned at Charlotte Correctional Institution ,  Punta Gorda, Florida 
 
 Tony Alvin Ables  (born December 28, 1954) is an  American   serial killer . Officially convicted of two murders, one of which occurred while he was still underage, Ables was later connected via  DNA  evidence to two other murders in 2006: of 84-year-old retiree Adeline McL

KeyboardInterrupt: 

In [108]:
murder_dataframe = pd.DataFrame.from_dict(murder_dict)
murder_dataframe.to_csv("scraped_murders.csv")

ValueError: arrays must all be same length